<a href="https://colab.research.google.com/github/nilperez/British-Airways-customer-reviews/blob/main/Task1/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
# Utilizo BeautifulSoup para obtener los reviews de SkyTrax
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []
title = []
date = []

for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Se crea la URL para recopilar enlaces de datos paginados
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Se recopilan datos HTML de esta página
    response = requests.get(url)

    # Se analiza el contenido
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("h2", {"class": "text_header"}):
        title.append(para.get_text())
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    for para in parsed_content.find_all("time", itemprop="datePublished"):
        date.append(para.get_text())

    print(f"   ---> {len(title)} total titles, {len(reviews)} total reviews, {len(date)} total dates")

Scraping page 1
   ---> 100 total titles, 100 total reviews, 100 total dates
Scraping page 2
   ---> 200 total titles, 200 total reviews, 200 total dates
Scraping page 3
   ---> 300 total titles, 300 total reviews, 300 total dates
Scraping page 4
   ---> 400 total titles, 400 total reviews, 400 total dates
Scraping page 5
   ---> 500 total titles, 500 total reviews, 500 total dates
Scraping page 6
   ---> 600 total titles, 600 total reviews, 600 total dates
Scraping page 7
   ---> 700 total titles, 700 total reviews, 700 total dates
Scraping page 8
   ---> 800 total titles, 800 total reviews, 800 total dates
Scraping page 9
   ---> 900 total titles, 900 total reviews, 900 total dates
Scraping page 10
   ---> 1000 total titles, 1000 total reviews, 1000 total dates


In [23]:
# Creo un DataFrame con las reviews scrapeadas
dataset = {
  'title': title,
  'reviews': reviews,
  'date': date
}
df = pd.DataFrame(dataset)
df.head()

,title,reviews,date
0,"""KLM is definitely a league over BA""",Not Verified | BA is not what it used to be! ...,11th March 2024
1,"""Service extremely inattentive""","✅ Trip Verified | BA First, it's not even the...",10th March 2024
2,"""worst business class experience""",✅ Trip Verified | The worst business class ex...,5th March 2024
3,"""it's truly awful for short-haul""",Not Verified | Quite possibly the worst busin...,4th March 2024
4,"""never be flying with BA again""",Not Verified | I will never be flying with BA...,4th March 2024


In [24]:
df.to_csv("/content/drive/MyDrive/data/Raw_BA_reviews.csv")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1000 non-null   object
 1   reviews  1000 non-null   object
 2   date     1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [37]:
# Muestro los tipos de datos de las columnas
print(df.dtypes)


title              object
reviews            object
date       datetime64[ns]
dtype: object


In [29]:
# Elimino lo que está antes de '|', ya que no es significativo
df['reviews'] = df['reviews'].str.split('|').str[1]

df['reviews'] = df['reviews'].str.strip()

In [38]:
# Elimino las comillas de los títulos
df['title'] = df['title'].apply(lambda x: x.replace('"', ''))

# Separo 'mes' y 'año' de 'date' y lo coloco en columnas separadas
df['month'] = pd.DatetimeIndex(df['date']).month
df['year'] = pd.DatetimeIndex(df['date']).year


In [42]:
df.head()


,title,reviews,month,year
0,KLM is definitely a league over BA,BA is not what it used to be! As much as I lik...,3,2024
1,Service extremely inattentive,"BA First, it's not even the best business clas...",3,2024
2,worst business class experience,The worst business class experience. Ground cr...,3,2024
3,it's truly awful for short-haul,Quite possibly the worst business class I have...,3,2024
4,never be flying with BA again,I will never be flying with BA again. This is ...,3,2024


In [41]:
# Elimino la columna 'date' que ya no es útil
df.drop('date', axis=1, inplace=True)

In [43]:
df.to_csv("Clean_BA_reviews.csv")